In [1]:
import findspark

In [2]:
findspark.init('/home/samet/spark-3.2.1-bin-hadoop3.2')

In [3]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [7]:
from pyspark.ml.classification import LogisticRegression

In [8]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

22/04/26 17:33:33 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [9]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [10]:
my_log_reg_model = LogisticRegression()

In [13]:
fitted_logreg = my_log_reg_model.fit(my_data)

In [14]:
log_summary = fitted_logreg.summary

In [17]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



/home/samet/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [19]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [20]:
lr_train, lr_test = my_data.randomSplit([0.7,0.3])

In [22]:
final_model = LogisticRegression()

In [23]:
fit_final = final_model.fit(lr_train)

In [24]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [25]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[35.1600103130600...|[0.99999999999999...|       0.0|
|  0.0|(692,[122,123,124...|[19.1058890139755...|[0.99999999496014...|       0.0|
|  0.0|(692,[123,124,125...|[35.5426300347981...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[22.1308379968913...|[0.99999999975526...|       0.0|
|  0.0|(692,[126,127,128...|[39.9464323252304...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[24.6678057087567...|[0.99999999998063...|       0.0|
|  0.0|(692,[127,128,129...|[25.8257134153555...|[0.99999999999391...|       0.0|
|  0.0|(692,[154,155,156...|[22.1486025240126...|[0.99999999975957...|       0.0|
|  0.0|(692,[155,156,180...|[38.4767330744024...|           [1.0,0.0]|       0.0|
|  0.0|(692,[181

/home/samet/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [27]:
my_eval = BinaryClassificationEvaluator()

In [28]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

/home/samet/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [29]:
my_final_roc

1.0